In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [5]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")

if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY is not set")

if google_api_key is None:
    raise ValueError("GOOGLE_API_KEY is not set")



# The API we know well
# I've updated this with the latest model, but it can take some time because it likes to think!
# Replace the model with gpt-4.1-mini if you'd prefer not to wait 1-2 mins

In [3]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to generate a code for algorithm like binary tree for live coding competition. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [4]:
print(messages)

[{'role': 'user', 'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to generate a code for algorithm like binary tree for live coding competition. Answer only with the question, no explanation.'}]


In [6]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)

How would you implement a binary tree in Python that includes methods for insertion, deletion, traversal (in-order, pre-order, post-order), and searching for a specific value, while also ensuring balanced height after each insertion?


In [7]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [ ]:
model_name = "gpt-5-mini"

openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=messages,
)
answer = response.choices[0].message.content

display(Markdown(answer))
answers.append(answer)
competitors.append(model_name)


In [ ]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.5-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
ollama = OpenAI(base_url="http://localhost:11434/v1")
model_name = "phi3:latest"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
print(competitors)
print(answers)

In [ ]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


In [25]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [26]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
print(judge)

In [27]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-5-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


In [ ]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")